## Assignment 3 - Fine Tuning a Language Model
###Classification (sentiment analysis) and Gradio UI 

Using Huggingface example - https://colab.research.google.com/github/huggingface/notebooks/blob/master/examples/text_classification.ipynb

In [1]:
import numpy as np

In [2]:
!pip install datasets transformers

     |████████████████████████████████| 270 kB 12.3 MB/s 
     |████████████████████████████████| 2.9 MB 44.6 MB/s 
     |████████████████████████████████| 243 kB 42.7 MB/s 
     |████████████████████████████████| 1.3 MB 40.7 MB/s 
     |████████████████████████████████| 56 kB 4.9 MB/s 
     |████████████████████████████████| 125 kB 39.7 MB/s 
     |████████████████████████████████| 636 kB 45.6 MB/s 
     |████████████████████████████████| 3.3 MB 40.5 MB/s 
     |████████████████████████████████| 895 kB 29.5 MB/s 
     |████████████████████████████████| 160 kB 51.1 MB/s 
     |████████████████████████████████| 271 kB 48.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
import transformers

print(transformers.__version__)

4.11.3


In [4]:
# Choosing a GLUE sentiment analysis task
task = "sst2"
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

## Get the Data

In [5]:
from datasets import load_dataset, load_metric

In [6]:
dataset = load_dataset('glue', task)
metric = load_metric('glue', task)

Downloading:   0%|          | 0.00/7.78k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.47k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/7.44M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/1.86k [00:00<?, ?B/s]

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})

In [8]:
dataset['train'][4]

{'idx': 4,
 'label': 0,
 'sentence': 'on the worst revenge-of-the-nerds clichés the filmmakers could dredge up '}

In [9]:
# Showing some examples from the dataset

import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [10]:
show_random_elements(dataset["train"])

,sentence,label,idx
0,the movie 's inescapable air of sleaziness,negative,47541
1,a freshness and modesty,positive,22096
2,", we hope it 's only acting .",positive,52027
3,"in the real world , an actor this uncharismatically beautiful would have a résumé loaded with credits like `` girl in bar # 3 . ''",negative,35704
4,that the film idiotically uses the website feardotcom.com or the improperly hammy performance from poor stephen rea,negative,65656
5,should have been a painless time-killer,negative,26437
6,"in fact , it 's quite fun in places .",positive,31285
7,"sure to give you a lot of laughs in this simple , sweet and romantic comedy",positive,30113
8,"you can almost see mendes and company getting together before a single frame had been shot and collectively vowing , ` this is going to be something really good . '",positive,26276
9,you might want to check it out,positive,8550


In [11]:
show_random_elements(dataset["test"])

,sentence,label,idx
0,"despite apparent motives to the contrary , it ends up being , like ( seinfeld 's ) revered tv show , about pretty much nothing .",positive,844
1,"if shayamalan wanted to tell a story about a man who loses his faith , why did n't he just do it , instead of using bad sci-fi as window dressing ?",positive,499
2,"serry does a fine job of capturing the climate of the times and , perhaps unwittingly , relating it to what is happening in america in 2002 .",positive,1061
3,"director kapur is a filmmaker with a real flair for epic landscapes and adventure , and this is a better film than his earlier english-language movie , the overpraised elizabeth .",positive,197
4,"an exciting and involving rock music doc , a smart and satisfying look inside that tumultuous world .",positive,1501
5,a dream cast of solid female talent who build a seamless ensemble .,positive,1568
6,"the actors do n't inhabit their roles -- they 're trapped by them , forced to change behavior in bizarre unjustified fashion and spout dialog that consists mostly of platitudes .",positive,1665
7,good ol' urban legend stuff .,positive,525
8,"as a singular character study , it 's perfect .",positive,1267
9,"but watching huppert , a great actress tearing into a landmark role , is riveting .",positive,473


### Preprocess the Data

In [12]:
from transformers import AutoTokenizer

In [13]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [14]:
tokenizer('this is a test sentence')

{'input_ids': [101, 2023, 2003, 1037, 3231, 6251, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [15]:
tokenizer('this is a test sentence', 'this sentence follows the first one!')

{'input_ids': [101, 2023, 2003, 1037, 3231, 6251, 102, 2023, 6251, 4076, 1996, 2034, 2028, 999, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [16]:
def preprocess_function(examples):
  return tokenizer(examples['sentence'], truncation=True)

In [17]:
preprocess_function(dataset['train'][5])

{'input_ids': [101, 2008, 1005, 1055, 2521, 2205, 13800, 2000, 7857, 2107, 23105, 3949, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [18]:
# Preprocess all data

encoded_dataset = dataset.map(preprocess_function, batched=True)

  0%|          | 0/68 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

## Fine Tuning

In [19]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

num_labels = 2
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'pre_classi

In [20]:
metric_name = "accuracy"
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)

In [21]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [22]:
validation_key = "validation"
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [23]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentence, idx.
***** Running training *****
  Num examples = 67349
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 21050


Epoch,Training Loss,Validation Loss,Accuracy
1,0.189100,0.311110,0.903670
2,0.128400,0.378690,0.904817
3,0.079200,0.384555,0.905963
4,0.070000,0.429580,0.901376
5,0.052900,0.504858,0.902523


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentence, idx.
***** Running Evaluation *****
  Num examples = 872
  Batch size = 16
Saving model checkpoint to distilbert-base-uncased-finetuned-sst2/checkpoint-4210
Configuration saved in distilbert-base-uncased-finetuned-sst2/checkpoint-4210/config.json
Model weights saved in distilbert-base-uncased-finetuned-sst2/checkpoint-4210/pytorch_model.bin
tokenizer config file saved in distilbert-base-uncased-finetuned-sst2/checkpoint-4210/tokenizer_config.json
Special tokens file saved in distilbert-base-uncased-finetuned-sst2/checkpoint-4210/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentence, idx.
***** Running Evaluation *****
  Num examples = 872
  Batch size = 16
Saving model checkpoint to distilbert-ba

TrainOutput(global_step=21050, training_loss=0.11122291476029965, metrics={'train_runtime': 2993.8291, 'train_samples_per_second': 112.48, 'train_steps_per_second': 7.031, 'total_flos': 3069326287719492.0, 'train_loss': 0.11122291476029965, 'epoch': 5.0})

## Gradio

In [24]:
!pip install gradio


     |████████████████████████████████| 3.4 MB 12.7 MB/s 
     |████████████████████████████████| 1.9 MB 37.8 MB/s 
     |████████████████████████████████| 206 kB 52.6 MB/s 
     |████████████████████████████████| 63 kB 2.4 MB/s 
     |████████████████████████████████| 3.5 MB 41.8 MB/s 
     |████████████████████████████████| 961 kB 28.6 MB/s 
  Created wheel for ffmpy: filename=ffmpy-0.3.0-py3-none-any.whl size=4710 sha256=bab64baaa8b1f8918f8c6136d1f65e4ffd83dd94e489f810692af98f29fe5354
  Stored in directory: /root/.cache/pip/wheels/13/e4/6c/e8059816e86796a597c6e6b0d4c880630f51a1fcfa0befd5e6
  Created wheel for flask-cachebuster: filename=Flask_CacheBuster-1.0.0-py3-none-any.whl size=3371 sha256=2c79834a79906a7ba00f4db6bc4abfa228df5e9370100609bae8f62e7f919566
  Stored in directory: /root/.cache/pip/wheels/28/c0/c4/44687421dab41455be93112bd1b0dee1f3c5a9aa27bee63708
Successfully built ffmpy flask-cachebuster


In [25]:
import gradio as gr

In [26]:
def sentiment_analysis(text):
    text = text.encode("ascii", errors="ignore").decode(
        "ascii"
    ) 
    results = sentiment(text)
    return results[0]["label"], round(results[0]["score"], 5)

In [27]:
gradio_ui = gr.Interface(
    fn=sentiment_analysis,
    title="Sentiment Analysis",
    description="Enter some text and see if we can gauge the sentiment correctly",
    inputs=gr.inputs.Textbox(lines=10, label="Paste some text here"),
    outputs=[
        gr.outputs.Textbox(label="Sentiment Label"),
        gr.outputs.Textbox(label="Sentiment Score"),
    ],
)

In [28]:
gradio_ui.launch()


Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
This share link will expire in 72 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted
Running on External URL: https://39072.gradio.app
Interface loading below...


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7860/',
 'https://39072.gradio.app')